In [1]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

In [2]:
import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# We import the simulation initial conditions and atmospheric density model
The files needed are:
* **debris_simulation_ic.pk** - created by the notebook 1
* **best_fit_density.pk** - created by the notebook 1b

In [3]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r_ic,v_ic,reentry_radius,to_satcat,satcat,debris = pkl.load(file)

* **r**: contains the initial position of all satellites to be simulated (SI units)
* **v**: contains the initial velocity of all satellites to be simulated (SI units)
* **radius**: contains all the estimated radii for the various objects (in meters)
* **to_satcat**: contains the indexes in the satcat of the corresponding r,v,radius entry
* **satcat**: the satcat
* **debris**: the corresponding pykep planets

In [4]:
with open("data/best_fit_density.pk", "rb") as file:
    best_x = pkl.load(file)

In [5]:
# We need to create an array containing all B*
BSTARS = []
for idx in to_satcat:
    BSTARS.append(float(satcat[idx]["BSTAR"]))
# We put the BSTAR in SI units
BSTARS = np.array(BSTARS) / pk.EARTH_RADIUS
# We remove negative BSTARS setting the value to zero in those occasions
BSTARS[BSTARS<0] = 0.

# We build the dynamical system to integrate

In [6]:
# This little helper returns the heyoka expression for the density using
# the results from the data interpolation
def compute_density(h, best_x):
    """
    Returns the heyoka expression for the atmosheric density in kg.m^3. 
    Input is the altitude in m. 
    (when we fitted km were used here we change as to allow better expressions)
    """
    p1 = np.array(best_x[:4])
    p2 = np.array(best_x[4:8]) / 1000
    p3 = np.array(best_x[8:]) * 1000
    retval = 0.
    for alpha,beta, gamma in zip(p1,p2, p3):
        retval += alpha*hy.exp(-(h-gamma)*beta)
    return retval

In [7]:
# Dynamical variables.
x,y,z,vx,vy,vz = hy.make_vars("x","y","z","vx","vy","vz")

# Constants.
GMe = pk.MU_EARTH
C20 = -4.84165371736e-4
C22 = 2.43914352398e-6
S22 = -1.40016683654e-6
Re = pk.EARTH_RADIUS

# Create Keplerian dynamics.
dyn = csc.dynamics.kepler(mu = GMe)

In [8]:
# Add the J2 terms.
magr2 = hy.sum_sq([x, y, z])
J2term1 = GMe*(Re**2)*np.sqrt(5)*C20/(2*magr2**(1./2))
J2term2 = 3/(magr2**2)
J2term3 = 15*(z**2)/(magr2**3)
fJ2x = J2term1*x*(J2term2 - J2term3)
fJ2y = J2term1*y*(J2term2 - J2term3)
fJ2z = J2term1*z*(3*J2term2 - J2term3)
dyn[3] = (dyn[3][0], dyn[3][1] + fJ2x)
dyn[4] = (dyn[4][0], dyn[4][1] + fJ2y)
dyn[5] = (dyn[5][0], dyn[5][1] + fJ2z)

In [9]:
# Add the Earth's C22 and S22 terms.
# This value represents the rotation of the Earth fixed system at t0
theta_g = (np.pi/180)*280.4606 #[rad] 
# This value represents the magnitude of the Earth rotation
nu_e = (np.pi/180)*(4.178074622024230e-3) #[rad/sec]

X =  x*hy.cos(theta_g + nu_e*hy.time) + y*hy.sin(theta_g + nu_e*hy.time)
Y = -x*hy.sin(theta_g + nu_e*hy.time) + y*hy.cos(theta_g + nu_e*hy.time)
Z = z

C22term1 = 5*GMe*(Re**2)*np.sqrt(15)*C22/(2*magr2**(7./2))
C22term2 = GMe*(Re**2)*np.sqrt(15)*C22/(magr2**(5./2))
fC22X = C22term1*X*(Y**2 - X**2) + C22term2*X
fC22Y = C22term1*Y*(Y**2 - X**2) - C22term2*Y
fC22Z = C22term1*Z*(Y**2 - X**2)

S22term1 = 5*GMe*(Re**2)*np.sqrt(15)*S22/(magr2**(7./2))
S22term2 = GMe*(Re**2)*np.sqrt(15)*S22/(magr2**(5./2))
fS22X = -S22term1*(X**2)*Y + S22term2*Y
fS22Y = -S22term1*X*(Y**2) + S22term2*X
fS22Z = -S22term1*X*Y*Z

fC22x = fC22X*hy.cos(theta_g + nu_e*hy.time) - fC22Y*hy.sin(theta_g + nu_e*hy.time)
fC22y = fC22X*hy.sin(theta_g + nu_e*hy.time) + fC22Y*hy.cos(theta_g + nu_e*hy.time)
fC22z = fC22Z

fS22x = fS22X*hy.cos(theta_g + nu_e*hy.time) - fS22Y*hy.sin(theta_g + nu_e*hy.time)
fS22y = fS22X*hy.sin(theta_g + nu_e*hy.time) + fS22Y*hy.cos(theta_g + nu_e*hy.time)
fS22z = fS22Z

dyn[3] = (dyn[3][0], dyn[3][1] + fC22x + fS22x)
dyn[4] = (dyn[4][0], dyn[4][1] + fC22y + fS22y)
dyn[5] = (dyn[5][0], dyn[5][1] + fC22z + fS22z)

In [10]:
# Adds the drag force.
magv2 = hy.sum_sq([vx, vy, vz])
magv = hy.sqrt(magv2)
# Here we consider a spherical Earth ... would be easy to account for the oblateness effect.
altitude = (hy.sqrt(magr2) - Re)
density = compute_density(altitude, best_x)
ref_density = 0.1570 / Re
fdrag = density / ref_density * hy.par[0] * magv
fdragx = - fdrag * vx
fdragy = - fdrag * vy
fdragz = - fdrag * vz
dyn[3] = (dyn[3][0], dyn[3][1] + fdragx)
dyn[4] = (dyn[4][0], dyn[4][1] + fdragy)
dyn[5] = (dyn[5][0], dyn[5][1] + fdragz)

# We setup the simulation

In [11]:
csc.set_logger_level_trace()

In [12]:
def remove_particle(idx, r_ic, v_ic, BSTARS,to_satcat, reentry_radius):
    r_ic = np.delete(r_ic, idx, axis=0)
    BSTARS = np.delete(BSTARS, idx, axis=0)
    v_ic = np.delete(v_ic, idx, axis=0)
    to_satcat = np.delete(to_satcat, idx, axis=0)
    reentry_radius = np.delete(reentry_radius, idx, axis=0)
    return r_ic, v_ic, BSTARS, to_satcat, reentry_radius

In [13]:
# Before starting we need to remove all particles inside our playing field
min_radius = pk.EARTH_RADIUS+150000.
inside_the_radius = np.where(np.linalg.norm(r_ic,axis=1) < min_radius)[0]
print("Removing orbiting objects:")
for idx in inside_the_radius:
    print(satcat[to_satcat[idx]]["OBJECT_NAME"], "-", satcat[to_satcat[idx]]["OBJECT_ID"])
r_ic, v_ic, BSTARS,to_satcat, reentry_radius = remove_particle(inside_the_radius, r_ic, v_ic, BSTARS,to_satcat, reentry_radius)

Removing orbiting objects:
LEMUR 2 ROCKETJONAH - 2017-071E
ISARA - 2017-071P
FREGAT DEB - 2011-037EM
STARLINK-1684 - 2020-070H
COSMOS 1408 DEB - 1982-092Z
COSMOS 1408 DEB - 1982-092AK
COSMOS 1408 DEB - 1982-092ES
COSMOS 1408 DEB - 1982-092FK
COSMOS 1408 DEB - 1982-092FY
COSMOS 1408 DEB - 1982-092GU
COSMOS 1408 DEB - 1982-092NA
COSMOS 1408 DEB - 1982-092PV
COSMOS 1408 DEB - 1982-092PW
COSMOS 1408 DEB - 1982-092RM
COSMOS 1408 DEB - 1982-092ACG
COSMOS 1408 DEB - 1982-092AQC
COSMOS 1408 DEB - 1982-092ARK
COSMOS 1408 DEB - 1982-092AXA
COSMOS 1408 DEB - 1982-092AXD
COSMOS 1408 DEB - 1982-092BDB
COSMOS 1408 DEB - 1982-092BFU
COSMOS 1408 DEB - 1982-092BKD


In [16]:
# Prepare the data in the shape expected by the simulation object.
ic_state = np.hstack([r_ic, v_ic, reentry_radius.reshape((r_ic.shape[0], 1))])
pars = BSTARS.reshape((r_ic.shape[0], 1))

# Uncommen these lines to save the txt files to be used in the cpp simulations
#np.savetxt("test_ic_19647.txt", ic_state.reshape((-1, 1))
#np.savetxt("test_par_19647.txt", pars)

In [ ]:
sim = csc.sim(ic_state,0.23 * 806.81,dyn=dyn,pars=pars, reentry_radius=min_radius)

# We run the simulation

In [16]:
# new_state  =deepcopy(ic_state)
# new_pars  =deepcopy(pars)
# new_to_satcat  =deepcopy(to_satcat)

rng = np.random


In [16]:
final_t = 365.25 * pk.DAY2SEC

pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(0.01, 0.1)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], pk.MU_EARTH)

            sim.state[idx,0:3] = r
            sim.state[idx,3:6] = v
            sim.state[idx,6] = size
    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        # We log on screen 
        print(satcat[to_satcat[pi]]["OBJECT_NAME"].strip() + ", " + satcat[to_satcat[pi]]["OBJECT_ID"].strip() + ", ", sim.time*pk.SEC2DAY, "REMOVED")
        # We remove the re-entered object and restart the simulation
        new_state = np.delete(sim.state,pi,axis=0)
        new_pars = np.delete(sim.pars,pi,axis=0)
        
        sim.set_new_state(new_state)
        sim.pars[:] = new_pars
        
        # new_r_ic = np.vstack((sim.x,sim.y,sim.z)).transpose()
        # new_v_ic = np.vstack((sim.vx,sim.vy,sim.vz)).transpose()
        # new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_reentry_radius = remove_particle(pi, new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_reentry_radius)
        # sim.set_new_state(new_r_ic[:,0],new_r_ic[:,1],new_r_ic[:,2],new_v_ic[:,0],new_v_ic[:,1],new_v_ic[:,2],new_reentry_radius, pars=[new_BSTARS])
pbar.close()
del pbar

  0%|          | 0/31557600.0 [00:00<?, ?it/s]

COSMOS 1408 DEB, 1982-092FH,  0.0011018046992662697 REMOVED
COSMOS 1408 DEB, 1982-092GL,  0.0041028438559408695 REMOVED
SL-4 R/B, 2006-061B,  0.005706346736241129 REMOVED
FREGAT DEB, 2011-037EQ,  0.007267201427187295 REMOVED
STARLINK-2025, 2021-009BM,  0.01815315899057631 REMOVED
COSMOS 1408 DEB, 1982-092ADW,  0.02952830918037765 REMOVED
COSMOS 2241, 1993-022A,  0.04331453508044084 REMOVED
OBJECT K, 2021-002K,  0.060801872464038606 REMOVED
RESURS O1 DEB, 1994-074BA,  0.07344824298679883 REMOVED
COSMOS 1408 DEB, 1982-092TY,  0.07895270149013031 REMOVED
FREGAT DEB, 2011-037NN,  0.07895517418611112 REMOVED
COSMOS 1408 DEB, 1982-092AHA,  0.11577314956600732 REMOVED
STARLINK-1906, 2020-074AC,  0.1579487707670265 REMOVED
OBJECT B, 2021-117B,  0.5273644864028021 REMOVED
COSMOS 2251 DEB, 1993-036AEX,  0.6732504658829894 REMOVED
COSMOS 1408 DEB, 1982-092FX,  1.1283422206984879 REMOVED
FALCON 9 DEB, 2020-055BM,  1.187062847177365 REMOVED
COSMOS 1408 DEB, 1982-092RY,  1.3262911429437225 REMOVED
C

In [17]:
final_t = 365.25 * pk.DAY2SEC

pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(0.01, 0.1)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], pk.MU_EARTH)

            sim.state[idx,0:3] = r
            sim.state[idx,3:6] = v
            sim.state[idx,6] = size
    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        # We log on screen 
        print(satcat[to_satcat[pi]]["OBJECT_NAME"].strip() + ", " + satcat[to_satcat[pi]]["OBJECT_ID"].strip() + ", ", sim.time*pk.SEC2DAY, "REMOVED")
        # We remove the re-entered object and restart the simulation
        new_state = np.delete(sim.state,pi,axis=0)
        new_pars = np.delete(sim.pars,pi,axis=0)
        
        sim.set_new_state(new_state)
        sim.pars[:] = new_pars
        
        # new_r_ic = np.vstack((sim.x,sim.y,sim.z)).transpose()
        # new_v_ic = np.vstack((sim.vx,sim.vy,sim.vz)).transpose()
        # new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_reentry_radius = remove_particle(pi, new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_reentry_radius)
        # sim.set_new_state(new_r_ic[:,0],new_r_ic[:,1],new_r_ic[:,2],new_v_ic[:,0],new_v_ic[:,1],new_v_ic[:,2],new_reentry_radius, pars=[new_BSTARS])
pbar.close()
del pbar

  0%|          | 0/31557600.0 [00:00<?, ?it/s]

COSMOS 1408 DEB, 1982-092FH,  0.001101804996265661 REMOVED
COSMOS 1408 DEB, 1982-092GL,  0.004102843891965073 REMOVED
SL-4 R/B, 2006-061B,  0.005706346736237456 REMOVED
FREGAT DEB, 2011-037EQ,  0.007267201428668006 REMOVED
STARLINK-2025, 2021-009BM,  0.018153158990584835 REMOVED
COSMOS 1408 DEB, 1982-092ADW,  0.029528243104741347 REMOVED
COSMOS 2241, 1993-022A,  0.04331453507422833 REMOVED
OBJECT K, 2021-002K,  0.06080187246401055 REMOVED
RESURS O1 DEB, 1994-074BA,  0.07344824309334752 REMOVED
FREGAT DEB, 2011-037NN,  0.0789510521099178 REMOVED
COSMOS 1408 DEB, 1982-092TW,  0.07895352180902593 REMOVED
COSMOS 1408 DEB, 1982-092AHA,  0.11577295604283491 REMOVED
STARLINK-1906, 2020-074AC,  0.15794831123395808 REMOVED
OBJECT B, 2021-117B,  0.5273460439998182 REMOVED
COSMOS 2251 DEB, 1993-036AEX,  0.6732365054153558 REMOVED
COSMOS 1408 DEB, 1982-092FX,  1.128301603795796 REMOVED
FALCON 9 DEB, 2020-055BM,  1.1870635028135692 REMOVED
COSMOS 1408 DEB, 1982-092RY,  1.3262372568571101 REMOVED
CO

KeyboardInterrupt: 